In [1]:
import pandas as pd
import numpy as np
import MyFunctionsList as mfs

In [2]:
df = pd.read_csv('raw_scraped_csv/joined_clubs_matches.csv')
df.head()

,pull_time,game_id,console,winning_club,h_club,h_club_id,h_goals,h_shots,h_shot_percent,h_passes_made,...,h_goals_per_match,h_goals_conceded_per_match,opp_club_id.1,opp_wins,opp_losses,opp_draws,opp_division,opp_tm_record,opp_goals_per_match,opp_goals_conceded_per_match
0,2023-03-21 05:02:18,15695788211,xboxone,Draw,Bang Average FC,1940614,1,6,0.17,95,...,2.63,0.75,15695788,56.0,8.0,14.0,1.0,WWWLLLLLDD,2.87,1.32
1,2023-03-21 05:02:18,1940614259,xboxone,Bang Average FC,Bang Average FC,1940614,5,13,0.38,92,...,2.63,0.75,18834,424.0,139.0,336.0,1.0,WWWWWLLLDD,2.55,2.31
2,2023-03-21 05:02:18,7612049113,xboxone,punjabi panther,Bang Average FC,1940614,1,7,0.14,99,...,2.63,0.75,7612049,57.0,16.0,34.0,2.0,WWWWWWWWLD,2.39,1.71
3,2023-03-21 05:02:18,19406141058,xboxone,Draw,Bang Average FC,1940614,1,11,0.09,154,...,2.63,0.75,176694,208.0,100.0,243.0,1.0,WWWLLLLDDD,1.89,2.07
4,2023-03-21 05:02:18,8369654865,xboxone,Bang Average FC,Bang Average FC,1940614,3,13,0.23,118,...,2.63,0.75,8369654,285.0,97.0,219.0,2.0,WWWWLLLLLD,2.31,2.05


In [3]:
# Generate and Store Columns which store the relative difference in division between opponents.
# If home team is 1 division higher, h_rel_div = 1, if 2 higher, = 2, if three lower, = -3, etc. 
df['h_rel_div'] = df['h_divison'] - df['opp_division']
df['opp_rel_div'] = df['opp_division'] - df['h_divison']

# Generates and stores total win % for the club. 
df['h_win_percent'] = df['h_wins'] / (df['h_wins'] + df['h_draws'] + df['h_losses'])
df['opp_win_percent'] = df['opp_wins'] / (df['opp_wins'] + df['opp_draws'] + df['opp_losses'])

# Generates and store recent form gpm per club. 
df['h_recent_win_average'] = df['h_tm_record'].apply(mfs.calculate_average)
df['opp_recent_win_average'] = df['opp_tm_record'].apply(mfs.calculate_average)

# Generates relative gpm stats. 
df['h_rel_gpm'] = df['h_goals_per_match'] - df['opp_goals_per_match']
df['opp_rel_gpm'] = df['opp_goals_per_match'] - df['h_goals_per_match']

# Generates relative gcpm stats. 
df['h_rel_gcpm'] = df['h_goals_conceded_per_match'] - df['opp_goals_conceded_per_match']
df['opp_rel_gcpm'] = df['opp_goals_conceded_per_match'] - df['h_goals_conceded_per_match']

# Stores total games played by club
df['h_games_played'] = df['h_wins'] + df['h_draws'] + df['h_losses']
df['opp_games_played'] = df['opp_wins'] + df['opp_draws'] + df['opp_losses']

In [4]:
df.columns

Index(['pull_time', 'game_id', 'console', 'winning_club', 'h_club',
       'h_club_id', 'h_goals', 'h_shots', 'h_shot_percent', 'h_passes_made',
       'h_pass_attempts', 'h_pass_percent', 'h_tackles_made',
       'h_tackle_attempts', 'h_tackle_percent', 'h_red_cards',
       'h_players_in_match', 'h_forward', 'h_midfielder', 'h_defender',
       'h_goalkeeper', 'h_any', 'h_season', 'h_round', 'opp_club',
       'opp_club_id', 'opp_goals', 'opp_shots', 'opp_shot_percent',
       'opp_passes_made', 'opp_pass_attempts', 'opp_pass_percent',
       'opp_tackles_made', 'opp_tackle_attempts', 'opp_tackle_percent',
       'opp_red_cards', 'opp_players_in_match', 'opp_forward',
       'opp_midfielder', 'opp_defender', 'opp_goalkeeper', 'opp_any',
       'opp_season', 'opp_round', 'h_club_id.1', 'h_wins', 'h_losses',
       'h_draws', 'h_divison', 'h_tm_record', 'h_goals_per_match',
       'h_goals_conceded_per_match', 'opp_club_id.1', 'opp_wins', 'opp_losses',
       'opp_draws', 'opp_division

In [5]:
# Given that the df provided effectively duplicate columns (one for the home team, and one for the away team), 
# we will strip split the match data in half (for home/away) and then merge it back together to provide 
# A single unified df 2x in lenghth. 

h_club_matches = df[['winning_club', 'h_club',
       'h_goals', 'h_shots',
       'h_pass_attempts', 'h_pass_percent',
       'h_tackle_attempts', 'h_tackle_percent', 'h_red_cards',
       'h_players_in_match', 'h_forward', 'h_midfielder', 'h_defender',
       'h_goalkeeper', 'h_any',
       'h_divison','h_goals_per_match',
       'h_goals_conceded_per_match','h_rel_div',
       'h_win_percent', 'h_recent_win_average',
       'h_rel_gpm','h_rel_gcpm', 'h_games_played']]

opp_club_matches = df[['winning_club', 'opp_club',
       'opp_goals', 'opp_shots',
       'opp_pass_attempts', 'opp_pass_percent',
       'opp_tackle_attempts', 'opp_tackle_percent', 'opp_red_cards',
       'opp_players_in_match', 'opp_forward', 'opp_midfielder', 'opp_defender',
       'opp_goalkeeper', 'opp_any',
       'opp_division','opp_goals_per_match',
       'opp_goals_conceded_per_match','opp_rel_div',
       'opp_win_percent', 'opp_recent_win_average',
       'opp_rel_gpm','opp_rel_gcpm', 'opp_games_played']]

In [6]:
print(len(h_club_matches.columns))
print(len(opp_club_matches.columns))

24
24


In [7]:
hcm_renamed = h_club_matches.rename(columns={'winning_club':'winner', 'h_club':'club',
       'h_goals':'goals', 'h_shots':'shots',
       'h_pass_attempts':'pass_attempts', 'h_pass_percent':'pass_percent',
       'h_tackle_attempts':'tackle_attempts', 'h_tackle_percent':'tackle_percent', 'h_red_cards':'red_cards',
       'h_players_in_match':'players_in_match', 'h_forward':'forwards', 'h_midfielder':'midfielders', 'h_defender':'defenders',
       'h_goalkeeper':'goalkeeper', 'h_any':'any',
       'h_divison':'division','h_goals_per_match':'goals_per_match',
       'h_goals_conceded_per_match':'goals_conceded_per_match','h_rel_div':'rel_div',
       'h_win_percent':'win_percent', 'h_recent_win_average':'recent_win_average',
       'h_rel_gpm':'rel_gpm','h_rel_gcpm':'rel_gcpm', 'h_games_played':'games_played'})

opp_renamed = opp_club_matches.rename(columns={'winning_club':'winner', 'opp_club':'club',
       'opp_goals':'goals', 'opp_shots':'shots',
       'opp_pass_attempts':'pass_attempts', 'opp_pass_percent':'pass_percent',
       'opp_tackle_attempts':'tackle_attempts', 'opp_tackle_percent':'tackle_percent', 'opp_red_cards':'red_cards',
       'opp_players_in_match':'players_in_match', 'opp_forward':'forwards', 'opp_midfielder':'midfielders', 'opp_defender':'defenders',
       'opp_goalkeeper':'goalkeeper', 'opp_any':'any',
       'opp_division':'division','opp_goals_per_match':'goals_per_match',
       'opp_goals_conceded_per_match':'goals_conceded_per_match','opp_rel_div':'rel_div',
       'opp_win_percent':'win_percent', 'opp_recent_win_average':'recent_win_average',
       'opp_rel_gpm':'rel_gpm','opp_rel_gcpm':'rel_gcpm', 'opp_games_played':'games_played'})

print(len(hcm_renamed))
print(len(opp_renamed))
unified_match_data = hcm_renamed.append(opp_renamed)
print(len(unified_match_data))
unified_match_data.head()

32899
32899
65798


,winner,club,goals,shots,pass_attempts,pass_percent,tackle_attempts,tackle_percent,red_cards,players_in_match,...,any,division,goals_per_match,goals_conceded_per_match,rel_div,win_percent,recent_win_average,rel_gpm,rel_gcpm,games_played
0,Draw,Bang Average FC,1,6,125,0.76,49,0.37,0,8,...,0,1.0,2.63,0.75,0.0,0.770588,2.0,-0.24,-0.57,510.0
1,Bang Average FC,Bang Average FC,5,13,120,0.77,29,0.45,0,8,...,0,1.0,2.63,0.75,0.0,0.770588,2.0,0.08,-1.56,510.0
2,punjabi panther,Bang Average FC,1,7,124,0.80,65,0.26,0,10,...,0,1.0,2.63,0.75,-1.0,0.770588,2.0,0.24,-0.96,510.0
3,Draw,Bang Average FC,1,11,195,0.79,41,0.39,0,11,...,0,1.0,2.63,0.75,0.0,0.770588,2.0,0.74,-1.32,510.0
4,Bang Average FC,Bang Average FC,3,13,145,0.81,43,0.23,0,10,...,0,1.0,2.63,0.75,-1.0,0.770588,2.0,0.32,-1.30,510.0


In [8]:
# Generates result column. Wins = 1, draws/losses = 0
unified_match_data['result'] = np.where(unified_match_data.club == unified_match_data.winner, 1, 0)

# There is a error in the data that occassionally reports > 1 any per team which isn't possible. 
# This is corrected by making any games with >= 1 any correct to having only 1 any. This makes it a binary 1/0 column. 
unified_match_data['any'] = np.where(unified_match_data['any'] >= 1, 1, 0)

# Similiar correction for keepers. You cannot have more than one keeper (the other may have been playing field positions)
unified_match_data['goalkeeper'] = np.where(unified_match_data['goalkeeper'] >= 1, 1, 0)


# This detects if a team has 0 passes made, which is only possible if a team lagged out. These games are removed. 
unified_match_data = unified_match_data[unified_match_data['pass_attempts'] != 0]

unified_match_data = unified_match_data.reset_index() 
unified_match_data = unified_match_data.drop(['club', 'index', 'winner'], axis=1)

In [9]:
len(unified_match_data)
unified_match_data = unified_match_data[np.isfinite(unified_match_data).all(1)]
len(unified_match_data)

65167

In [10]:
unified_match_data.to_csv('models/joined_model_ready_match_data.csv', index=False)